In [82]:
from __future__ import division
import numpy as np
import os, sys
import matplotlib
#matplotlib.use('Agg')
%matplotlib nbagg
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import csv


In [83]:
dhcp_data = []
ctr = 0
with open('data/hashed_mac_dhcp_lease_20170207_20170212.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if ctr > 0:
            dhcp_data.append([ctr]+row[1:])
        ctr += 1

In [84]:
import datetime
import time 

def get_timestamp(s):
    dt = datetime.datetime.strptime(s, "%Y/%m/%d %H:%M:%S")
    return time.mktime(dt.timetuple()) + (dt.microsecond / 1000000.0)
    

In [85]:
dhcp_rdd = sc.parallelize(dhcp_data).map(lambda s: tuple(s[:12]+[get_timestamp(s[12]), get_timestamp(s[13])]+s[14:]))
ip_2_mac = (dhcp_rdd
            .map(lambda s: ((s[9], s[10])))
            .groupByKey()
            .map(lambda s: (s[0], set(s[1])))
           )
mac_2_ip = (dhcp_rdd
            .map(lambda s: ((s[10], s[9])))
            .groupByKey()
            .map(lambda s: (s[0], set(s[1])))
           )

In [86]:
print "Total unique IP addresses:", dhcp_rdd.map(lambda s: s[9]).distinct().count()
print "Total unique MAC addresses:", dhcp_rdd.map(lambda s: s[10]).distinct().count()
print "Total IP addresses used by more than one mac", ip_2_mac.filter(lambda s: len(s[1])>1).count()
print "Total Mac addresses using more than IP addresses", mac_2_ip.filter(lambda s: len(s[1])>1).count()

Total unique IP addresses: 6425
Total unique MAC addresses: 6256
Total IP addresses used by more than one mac 0
Total Mac addresses using more than IP addresses 165


In [87]:
import pickle
with open('data/mac_2_ip_dhcp.pickle', 'w') as f:
    pickle.dump(mac_2_ip.collect(), f)

In [89]:
ip_2_mac_dict = dict((x,y) for x,y in ip_2_mac.collect())

In [95]:
print len(ip_2_mac_dict.keys())

['10.8.155.18', '10.8.76.67', '10.9.233.192', '10.8.68.36', '10.24.104.79', '10.8.31.3', '10.8.165.173', '10.8.165.172', '10.8.241.228', '10.8.165.170', '10.9.95.130', '10.24.125.115', '10.9.188.45', '10.9.110.8', '10.9.239.249', '10.9.195.87', '10.9.244.17', '10.8.141.119', '10.25.220.123', '10.25.220.124', '10.8.44.69', '10.8.244.124', '10.9.242.11', '10.9.50.249', '10.9.50.241', '10.8.87.95', '10.8.50.90', '10.8.87.96', '10.8.50.96', '10.8.190.53', '10.8.87.98', '10.8.26.223', '10.9.86.85', '10.8.26.229', '10.8.161.246', '10.9.28.10', '10.24.245.1', '10.9.45.130', '10.9.28.18', '10.9.35.245', '10.8.153.12', '10.9.54.72', '10.8.124.216', '10.25.154.213', '10.9.236.55', '10.8.151.11', '10.8.43.124', '10.25.33.249', '10.8.248.204', '10.8.86.240', '10.8.245.46', '10.8.38.108', '10.9.81.64', '10.9.163.148', '10.9.187.157', '10.8.234.16', '10.9.49.234', '10.8.252.104', '10.8.123.35', '10.9.73.246', '10.9.73.243', '10.24.110.202', '10.9.73.249', '10.8.240.105', '10.8.103.107', '10.8.21.85'

In [98]:
print ip_2_mac_dict['10.9.225.184']

set(['C0:33:5E:28:F:61'])


In [97]:
print mac_2_ip_dict['4C:66:41:F:F4:C7']

set(['10.8.252.6'])


In [99]:
import pickle
with open('data/ip_2_mac_dhcp.pickle', 'w') as f:
    pickle.dump(ip_2_mac.collect(), f)